# Fastq -> expression matrix with kb

## Install dependencies

Mount GoogleDrive if not already monted

In [0]:
from google.colab import drive
from os.path import join
ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/Mp_scRNA-seq'       # path to your project on Drive
drive.mount(ROOT)           # we mount the drive at /content/drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Install required packages

In [0]:
!pip install scanpy loompy scvelo anndata velocyto python-igraph louvain 

  Cloning https://github.com/pachterlab/kb_python (to revision count-kite) to /tmp/pip-req-build-pxxrw43f
  Running command git clone -q https://github.com/pachterlab/kb_python /tmp/pip-req-build-pxxrw43f
  Running command git checkout -b count-kite --track origin/count-kite
  Switched to a new branch 'count-kite'
  Branch 'count-kite' set up to track remote branch 'count-kite' from 'origin'.
     |████████████████████████████████| 7.2MB 1.7MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 194kB 56.1MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 204kB 61.7MB/s 
     |████████████████████████████████| 3.2MB 35.8MB/s 
     |████████████████████████████████| 92kB 13.9MB/s 
     |████████████████████████████████| 2.9MB 55.8MB/s 
     |████████████████████████████████| 7.1MB 46.2MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 51kB 7.7MB

In [0]:
!pip install git+https://github.com/pachterlab/kb_python@count-kite

  Cloning https://github.com/pachterlab/kb_python (to revision count-kite) to /tmp/pip-req-build-m6yhw0w0
  Running command git clone -q https://github.com/pachterlab/kb_python /tmp/pip-req-build-m6yhw0w0
  Running command git checkout -b count-kite --track origin/count-kite
  Switched to a new branch 'count-kite'
  Branch 'count-kite' set up to track remote branch 'count-kite' from 'origin'.
  Created wheel for kb-python: filename=kb_python-0.24.4-cp36-none-any.whl size=89699215 sha256=124c3e7f5a05699408fbdc77e7204eb04a5e61e0ce5c0d18e8a55c9bcb0e85c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-6j6t8x8q/wheels/8e/56/56/c89223de74af26792675e82f4bb5223e7cf0d653a33038e34c
Successfully built kb-python


Change to correct directory if not already there

In [0]:
cd drive/My\ Drive/Mp_scRNA-seq

/content/drive/My Drive/Mp_scRNA-seq


## Make index files for Mp_JGI_5.1

Create standard index file

In [0]:
!kb ref -i index/mp_51_index.idx -g index/t2g.txt -f1 index/cdna.fa --workflow standard \
genome/MpTak1v5.1.fasta \
genome/MpTak1v51.gtf


Create scVelo compatible index

In [0]:
!kb ref -i velo_index/mp_51_velo_index.idx -g velo_index/t2g.txt -f1 velo_index/cdna.fa -f2 velo_index/intron.fa -c1 velo_index/cdna_t2c.txt -c2 velo_index/intron_t2c.txt --workflow lamanno \
genome/MpTak1v5.1.fasta \
genome/MpTak1v51.gtf

[2020-03-25 12:41:05,909]    INFO Preparing index/MpTak1v5.1.fasta, index/MpTak1v51.gtf
[2020-03-25 12:41:05,911]    INFO Sorting index/MpTak1v5.1.fasta to /content/drive/My Drive/Mp_scRNA-seq/tmp/tmpfnsoe7nc
An exception occurred
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kb_python/main.py", line 703, in main
    COMMAND_TO_FUNCTION[args.command](parser, args, temp_dir=temp_dir)
  File "/usr/local/lib/python3.6/dist-packages/kb_python/main.py", line 131, in parse_ref
    temp_dir=temp_dir
  File "/usr/local/lib/python3.6/dist-packages/kb_python/ref.py", line 575, in ref_lamanno
    fasta_path, get_temporary_filename(temp_dir)
  File "/usr/local/lib/python3.6/dist-packages/kb_python/ref.py", line 51, in sort_fasta
    return fasta.sort(out_path)
  File "/usr/local/lib/python3.6/dist-packages/kb_python/fasta.py", line 126, in sort
    line = f.readline()
OSError: [Errno 5] Input/output error
[2020-03-25 12:41:08,770]   ERROR An exception occurred
T

## Pseudoallignment

Standard allignment h5ad output file

In [0]:
!kb count --h5ad -i index/mp_51_index.idx -g index/t2g.txt -x 10xv2 -o tak_51 --filter bustools -t 2 \
fastq/SIGAC11_S1_L008_R1_001.fastq.gz \
fastq/SIGAC11_S1_L008_R2_001.fastq.gz \
fastq/SIGAC11_S2_L008_R1_001.fastq.gz \
fastq/SIGAC11_S2_L008_R2_001.fastq.gz \
fastq/SIGAC11_S3_L008_R1_001.fastq.gz \
fastq/SIGAC11_S3_L008_R2_001.fastq.gz \
fastq/SIGAC11_S4_L008_R1_001.fastq.gz \
fastq/SIGAC11_S4_L008_R2_001.fastq.gz 

Scvelo allignment h5ad output file 

In [0]:
kb count --h5ad -i velo_index/mp_51_velo_index.idx -g velo_index/t2g.txt -x 10xv2 -o velo_tak_51 -c1 velo_index/cdna_t2c.txt -c2 intron_t2c.txt --filter bustools -t 2 \
fastq/SIGAC11_S1_L008_R1_001.fastq.gz \
fastq/SIGAC11_S1_L008_R2_001.fastq.gz \
fastq/SIGAC11_S2_L008_R1_001.fastq.gz \
fastq/SIGAC11_S2_L008_R2_001.fastq.gz \
fastq/SIGAC11_S3_L008_R1_001.fastq.gz \
fastq/SIGAC11_S3_L008_R2_001.fastq.gz \
fastq/SIGAC11_S4_L008_R1_001.fastq.gz \
fastq/SIGAC11_S4_L008_R2_001.fastq.gz 